# Project final # 
#### Akbar Ismatullayev 17376021, Jairoop Brar, Jaime Keith, Ian Yoon ###

In [ ]:
library(tidyverse)
library(lubridate)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

#### Reading the data for analysing

In [ ]:
players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

head(sessions)
head(players)

# quick sense-check on dimensions
dim(players)
dim(sessions)

## Introduction

#### About the data

The data this report will be analyzing comes from The Pacific Laboratory for Artificial Intelligence right here at UBC. They set up a minecraft server called PLAICraft and record play sessions with the goal of understanding how people play video games in order to advance artificial intelligence. They collected a set of data on the players on their server and are interested to know what kinds of players are the most likely to contribute more data so they can target them in their recruitment efforts. 


#### Data Description: ###


In [ ]:
glimpse(players)
glimpse(sessions)

player.csv contains demographic and information about each unique player. Each row represents one player, so there are 196 unique players.
What we also noticed that the formatting style of the column names are inconsistent, some are camelcase some are with an underscore.
For the variables:
* Experience is a character variable, with some classifactions like pro, veteran etc. It looks like an ordinal feature which means there is a orderning.
* subscribe is a boolean, saying if the player is subscribed to newsletter.
* hashed_email is a character variable which is unique for each player
* played_hours is a double variable, which says how many hours each player plays, however what we noticed is that there are players with zero play hours time, so they only created the account but no playing time, so we have to be carefull of that.
* name is a character variable, which is not unique and it is not usefull


session.csv contains records of all gameplay sessions for every player which can be connected through their 'hashed_email', also the variables as  their timestamps and duration which we can calculate. Each row corresponds to one gameplay session. Since there are 1535 rows which are more than the amount of playes, this suggest that there are duplicates of the hashed_email in the dataset so players are gaming more than once on their server. So we have to be carefull of that.
For the variables:
* hashed_email is a character variable which you can link to the player.
* start_time is a character variable saying when the session started.
* end_time is a character variable saying when the session ended.
* both orginal_start_time and orginal_end_time are the same or almost the same so I will remove them.

In order to target future players on how much data they are expected to contribute, we needed to analyze the player data set to see if any of these variables are able to predict how many hours someone will play on the server.


#### Summary statistics

In [ ]:
summary(players)
summary(sessions)


Looking at the summary, we notice that there are empty columns in age (players) and orginal_end_time (session). Which we need to be carefull. For the subscribe variable we notice it is very skewed, most players are subscribed so if we want to classify a player if it is subscribed we need to be carefull because it is imbalanced so it can be tricky.

##### Potential issues in short #####
* age and orginal_end_time has na values
* experience can be made as an ordinal feature so beginner = 1, intermediate = 2, advanced = 3, pro = 4.
* some players dont have play time so they dont have any game session.
* both original_start_time and original_end_time are almost the same or the same
* subscribe is very imbalanced so there more true values than false values which can mislead a classification model.

#### Project Statement

The question this report is investigating is: 

*Can age predict experience level and/or the amount of hours played on the server and if so which specific age groups have the highest experience level and hour played?*

To answer this, the following variables do support the research question:

##### * Age — the main predictor variable.

##### * Experience — an ordinal outcome measure of player skill.

##### * Played_hours — a continuous measure representing total time spent playing.

Additional variables such as start_time, end_time, and duration_min from the sessions dataset can be used to validate play time records, although they are not essential for answering the primary question.

Variables including gender, subscribe and hashedEmail do not contribute to the research question and are therefore excluded from further analysis.



## Methods & Results

## Discussion

In [ ]:
# variable dictionary (just to keep track of what's what) 

var_dict <- tibble(
  table    = c(rep("players", 7), rep("sessions", 5)),
  variable = c(
    "experience","subscribe","hashedEmail","played_hours","name","gender","Age",
    "hashedEmail","start_time","end_time","original_start_time","original_end_time"
  ),
  type = c(
    "factor","logical","id","numeric","string","factor","numeric",
    "id","string","string","numeric","numeric"
  ),
  meaning = c(
    "Experience tier","Has subscription","Unique player key","Total lifetime hours",
    "Player name","Self-reported gender","Age in years",
    "Unique player key","Session start (d/m/Y H:M)","Session end (d/m/Y H:M)",
    "Start epoch (ms)","End epoch (ms)"
  )
)

var_dict

# convert session timestamps to usable datetimes --------------------------
# The start/end times are strings like "14/05/2024 19:22" so I parsed them
# Then computed the playtime length for each session in minutes

sessions <- sessions |>
  mutate(
    start_time = dmy_hm(start_time, tz = "UTC"),
    end_time   = dmy_hm(end_time,   tz = "UTC"),
    duration_min = as.numeric(difftime(end_time, start_time, units = "mins"))
  )

# summarise total playtime per player 
# For each hashedEmail, we want:
# - total minutes played
# - average session length
# - number of sessions actually logged

by_player <- sessions |>
  group_by(hashedEmail) |>
  summarise(
    minutes  = sum(duration_min, na.rm = TRUE),
    avg_min  = mean(duration_min, na.rm = TRUE),
    sessions = sum(!is.na(duration_min)),
    .groups = "drop"
  )

# join the playtime summary onto the players table 
# If a player has no sessions, I filled their minutes/avg/sessions with zero.

dat <- players |>
  left_join(by_player, by = "hashedEmail") |>
  mutate(
    minutes  = replace_na(minutes,  0),
    avg_min  = replace_na(avg_min,  0),
    sessions = replace_na(sessions, 0)
  )

dat |>
  select(hashedEmail, experience, subscribe, Age, minutes, sessions) |>
  head()

# numeric summaries for the players table 
# A quick overview of the numeric columns (Age, played_hours, etc.)

players_num_summary <- players |>
  select(where(is.numeric)) |>
  pivot_longer(everything(), names_to = "variable", values_to = "value") |>
  group_by(variable) |>
  summarise(
    n    = sum(!is.na(value)),
    mean = mean(value, na.rm = TRUE),
    sd   = sd(value,   na.rm = TRUE),
    min  = min(value,  na.rm = TRUE),
    p25  = quantile(value, 0.25, na.rm = TRUE),
    med  = median(value,   na.rm = TRUE),
    p75  = quantile(value, 0.75, na.rm = TRUE),
    max  = max(value,  na.rm = TRUE),
    .groups = "drop"
  )

players_num_summary

# summary of session durations 
# Same type of summary but just for the session length variable.

duration_summary <- sessions |>
  summarise(
    n    = sum(!is.na(duration_min)),
    mean = mean(duration_min, na.rm = TRUE),
    sd   = sd(duration_min,   na.rm = TRUE),
    min  = min(duration_min,  na.rm = TRUE),
    p25  = quantile(duration_min, 0.25, na.rm = TRUE),
    med  = median(duration_min,   na.rm = TRUE),
    p75  = quantile(duration_min, 0.75, na.rm = TRUE),
    max  = max(duration_min,  na.rm = TRUE)
  )

duration_summary

# plot: age vs total minutes 
# We log-transform (minutes + 1) because playtime is extremely skewed,
# and we colour by subscription status just to see if subs behave differently.

dat |>
  ggplot(aes(x = Age,
             y = log10(minutes + 1),
             colour = subscribe)) +
  geom_point(alpha = 0.8) +
  scale_colour_manual(
    values = c("FALSE" = "black", "TRUE" = "blue"),
    name = "Subscribed"
  ) +
  labs(
    x = "Age (years)",
    y = "Total minutes played (log10 scale)",
    title = "Age vs total minutes"
  ) +
  theme_bw()


In [ ]:
# 1. How are player ages distributed? 
# Gives a feel for who is actually on the server.

dat |>
  ggplot(aes(x = Age)) +
  geom_histogram(binwidth = 2, boundary = 0, closed = "left") +
  labs(
    x = "Age (years)",
    y = "Number of players",
    title = "Distribution of player ages"
  ) +
  theme_bw()


# 2. Raw total minutes (super skewed, but good to see once) 
# Shows just how locked in some players are.

dat |>
  ggplot(aes(x = minutes)) +
  geom_histogram(bins = 40) +
  labs(
    x = "Total minutes played",
    y = "Number of players",
    title = "Total minutes played (raw scale)"
  ) +
  theme_bw()


# 3. Total minutes on a log scale 
# Same thing but easier to read the bulk of players.

dat |>
  ggplot(aes(x = log10(minutes + 1))) +
  geom_histogram(bins = 30) +
  labs(
    x = "log10(total minutes + 1)",
    y = "Number of players",
    title = "Total minutes played (log10 scale)"
  ) +
  theme_bw()


# 4. Playtime vs age, coloured by subscription 

dat |>
  ggplot(aes(x = Age,
             y = log10(minutes + 1),
             colour = subscribe)) +
  geom_point(alpha = 0.8) +
  scale_colour_manual(
    values = c("FALSE" = "black", "TRUE" = "blue"),
    name = "Subscribed"
  ) +
  labs(
    x = "Age (years)",
    y = "Total minutes played (log10 scale)",
    title = "Age vs total minutes"
  ) +
  theme_bw()


# 5. Do subscribers actually play more? hint: yes lol
# Boxplot of playtime split by subscription status.

dat |>
  ggplot(aes(x = subscribe,
             y = log10(minutes + 1),
             fill = subscribe)) +
  geom_boxplot(alpha = 0.6, outlier.alpha = 0.6) +
  scale_fill_manual(values = c("FALSE" = "grey70", "TRUE" = "skyblue")) +
  labs(
    x = "Subscribed?",
    y = "Total minutes played (log10 scale)",
    title = "Playtime by subscription status"
  ) +
  theme_bw() +
  theme(legend.position = "none")


# 6. Playtime by experience tier 

dat |>
  ggplot(aes(x = experience,
             y = log10(minutes + 1),
             fill = experience)) +
  geom_boxplot(alpha = 0.7, outlier.alpha = 0.6) +
  labs(
    x = "Experience tier",
    y = "Total minutes played (log10 scale)",
    title = "Playtime by experience tier"
  ) +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 30, hjust = 1),
        legend.position = "none")


# 7. What do individual sessions look like? 
# Distribution of single-session lengths in minutes.

sessions |>
  ggplot(aes(x = duration_min)) +
  geom_histogram(bins = 40) +
  labs(
    x = "Session duration (minutes)",
    y = "Number of sessions",
    title = "Distribution of session lengths"
  ) +
  theme_bw()


# 8. Zoomed-in view of “normal” sessions 
# Cuts off extreme marathons so we can see the typical range better.

sessions |>
  filter(duration_min <= 180) |>  # keep sessions up to 3 hours
  ggplot(aes(x = duration_min)) +
  geom_histogram(binwidth = 10, boundary = 0) +
  labs(
    x = "Session duration (minutes)",
    y = "Number of sessions",
    title = "Session lengths up to 3 hours"
  ) +
  theme_bw()
